In [8]:
import datetime
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error 
import joblib
import warnings
warnings.filterwarnings('ignore')

In [9]:
total = pd.read_csv('./make_data/0D_temp_1988_서울.csv')
test = pd.read_csv('./make_data/1_final_test.csv')

# 기상청 10일치 예보 데이터 
weather_10 = pd.read_csv("./make_data/0D_weather_10_float.csv")
weather_10.columns=['일시','temp']

In [10]:
total = total[(total.month==1)|(total.month==2)|(total.month==3)]
train_years = [1991, 1992, 1993, 1997, 1998, 1999] 
val_years = [2018]

train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

features = ['lunMonth', 'lunDay', 'time', 'month']

train_x = train[features]
train_y = train['temp']
val_x = val[features]
val_y = val['temp']
test_x = test[features] 
test_sample10_y = weather_10['temp']

train_x = train_x.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)
val_x = val_x.reset_index(drop=True)
val_y = val_y.reset_index(drop=True)
test_x = test_x.reset_index(drop=True)

In [11]:
model_lgb = lgb.LGBMRegressor(objective='regression', 
                              learning_rate=0.17, max_depth = 11, 
                              n_estimators= 135, num_leaves=110)

model_lgb.fit(train_x, train_y)

#모델 저장
joblib.dump(model_lgb, './models/temp_lgb_model.pkl')

prediction = model_lgb.predict(test_x)
test['temp']=prediction

In [12]:
pred_df = pd.DataFrame(test.loc[0:239, 'temp'])
sample_df = pd.DataFrame(test_sample10_y)
maes = mean_absolute_error(sample_df, pred_df)
maes

2.244028252392504

### testset에 기상청 10일치 예보 덮어쓰기

In [13]:
forecast = weather_10['temp'][:240].to_numpy()
for idx in range(7):
            test.loc[0+(2160*idx):239+(2160*idx), 'temp'] = forecast

In [14]:
test.to_csv('./make_data/2_final_test.csv',index=False)